# SETC - Cancellation Predictive Model 

## Import the Packages 

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd


# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import statements required for Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# for warnings 
import warnings
warnings.filterwarnings('ignore')



from sklearn.metrics import accuracy_score, log_loss
from imblearn.over_sampling import SMOTE
#import xgboost

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data 

In [ ]:
booking = pd.read_excel('Booking_April Full.xlsx') # To read the Booking data excel file 
cancel = pd.read_excel('Cancellation_Apr full.xlsx') # To read the cancel data excel file 
consol = pd.read_excel('JBDL Consol Apr.xlsx') # To read the consol data excel file 

In [48]:
# The Shape of the each data
print(booking.shape)
print(cancel.shape)
print(consol.shape)

(288643, 37)
(20139, 21)
(413679, 25)


In [46]:
consol.isna().values.any() # To find out NaN values, TRUE = yes, False = No

True

In [47]:
cancel.isnull().values.any() # To find out Null values TRUE = yes, False = No

True

# 1. Creat new features from the data set

## Days_btw_booking_journy
 Number of Days between booking date and journy date 

In [ ]:
booking_dt = pd.to_datetime(booking['BK_DATE']) # convert to date time format 
journy_dt = pd.to_datetime(booking['DT_JR'])

In [49]:
booking['Days_btw_booking_journy'] = journy_dt - booking_dt 
booking['Days_btw_booking_journy'] = booking['Days_btw_booking_journy'].dt.days
booking['Days_btw_booking_journy'] = booking['Days_btw_booking_journy'].fillna(0.0).astype(int) # replace missing values with 0.0
booking.head()

,BK_DATE,BK_TIME,COUNTER,R_CENTRE,BKTYPE,TRIPCODE,DeptTime,ROUTE,BKPNRNO,BKTKTNO,...,STUCODE,REGION,TOTTKT,USERNAME,USERTYPE,UserCORP,OBNO,SER_FARE,PG_FEE,Days_btw_booking_journy
0,2017-04-14,1970-01-01 01:07:00,ONLTC,ONLINE BUS STATION,E,2100CHEMADAC,1970-01-01 21:50:00,137AC,T19882990,56221416,...,S,MAD,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8790058,0,0,1
1,2017-04-14,1970-01-01 01:07:00,ONLTC,ONLINE BUS STATION,E,2230TRICHEKK02S,1970-01-01 22:30:00,908UA,B19758929,56221417,...,K,KUM,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8719270,0,0,2
2,2017-04-14,1970-01-01 01:10:00,ONLTC,ONLINE BUS STATION,E,2300CHESALAC,1970-01-01 23:50:00,422AC,T19889507,56221418,...,S,TRI,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8793460,10,7,0
3,2017-04-14,1970-01-01 01:10:00,ONLTC,ONLINE BUS STATION,E,1830TUTBAN,1970-01-01 18:30:00,872UD,T19889506,56221419,...,S,TIU,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8793459,10,12,2
4,2017-04-14,1970-01-01 01:11:00,ONLTC,ONLINE BUS STATION,E,2045CHEKRUAC,1970-01-01 21:35:00,429AC,T19823625,56221420,...,S,CHE,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8755994,0,0,0


## Cancelled Ticket 
The cancelled tickets on the basis of PNR number . This is a target Variable

In [ ]:
cancel = cancel.rename(index=str, columns={"PnrNoCanc": "BKPNRNO"}) # rename the PnrNoCanc to BKPNRNO

In [ ]:
height = len(cancel)  # create the new columns in consol called "cancelled_Ticket"
cancel['cancelled_Ticket'] = np.ones( (height,1) )
gh = cancel['cancelled_Ticket'].to_frame()
gh = gh.set_index(cancel['BKPNRNO'])
data_1= booking.join(gh, on='BKPNRNO') # join this column in booking data on the basis of BKPNRNO
data_1.cancelled_Ticket.fillna(0, inplace=True) #  missing value states the non canclled tickets and reaplce them with zero
data_1["cancelled_Ticket"] = data_1["cancelled_Ticket"].fillna(0.0).astype(int)

In [50]:
data_1.head()

,BK_DATE,BK_TIME,COUNTER,R_CENTRE,BKTYPE,TRIPCODE,DeptTime,ROUTE,BKPNRNO,BKTKTNO,...,REGION,TOTTKT,USERNAME,USERTYPE,UserCORP,OBNO,SER_FARE,PG_FEE,Days_btw_booking_journy,cancelled_Ticket
0,2017-04-14,1970-01-01 01:07:00,ONLTC,ONLINE BUS STATION,E,2100CHEMADAC,1970-01-01 21:50:00,137AC,T19882990,56221416,...,MAD,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8790058,0,0,1,1
1,2017-04-14,1970-01-01 01:07:00,ONLTC,ONLINE BUS STATION,E,2230TRICHEKK02S,1970-01-01 22:30:00,908UA,B19758929,56221417,...,KUM,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8719270,0,0,2,1
2,2017-04-14,1970-01-01 01:10:00,ONLTC,ONLINE BUS STATION,E,2300CHESALAC,1970-01-01 23:50:00,422AC,T19889507,56221418,...,TRI,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8793460,10,7,0,0
3,2017-04-14,1970-01-01 01:10:00,ONLTC,ONLINE BUS STATION,E,1830TUTBAN,1970-01-01 18:30:00,872UD,T19889506,56221419,...,TIU,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8793459,10,12,2,0
4,2017-04-14,1970-01-01 01:11:00,ONLTC,ONLINE BUS STATION,E,2045CHEKRUAC,1970-01-01 21:35:00,429AC,T19823625,56221420,...,CHE,1,ONLINE USER,Operator,STATE EXPRESS TRANSPORT CORPORATION TAMILNADU ...,OB8755994,0,0,0,1


# <brc>

# 2. Data Cleaning and Dropping

In [51]:
data_1.shape

(289346, 38)

In [ ]:
final_data= data_1.drop(['BK_DATE','BK_TIME', 'TRIPCODE', 'DeptTime', 'BKPNRNO', 'BKTKTNO', 'DT_JR',
                                   'BKSEAT','TOTTKT','UserCORP','OBNO'], axis=1) # Drop the unwanted features or coulmns 

In [52]:
final_data.head()

,COUNTER,R_CENTRE,BKTYPE,ROUTE,FROMPLACE,TOPLACE,AduLT,CHILD,AdultChildcount,TRNTYP,...,NOT_COLL,CON,STUCODE,REGION,USERNAME,USERTYPE,SER_FARE,PG_FEE,Days_btw_booking_journy,cancelled_Ticket
0,ONLTC,ONLINE BUS STATION,E,137AC,CHE,MAD,0,0,0,A,...,0,0,S,MAD,ONLINE USER,Operator,0,0,1,1
1,ONLTC,ONLINE BUS STATION,E,908UA,TRI,CHE,0,0,0,A,...,0,0,K,KUM,ONLINE USER,Operator,0,0,2,1
2,ONLTC,ONLINE BUS STATION,E,422AC,CHE,SAL,1,0,1,B,...,0,0,S,TRI,ONLINE USER,Operator,10,7,0,0
3,ONLTC,ONLINE BUS STATION,E,872UD,TUT,BAN,1,0,1,B,...,0,0,S,TIU,ONLINE USER,Operator,10,12,2,0
4,ONLTC,ONLINE BUS STATION,E,429AC,CHE,KRU,0,0,0,A,...,0,0,S,CHE,ONLINE USER,Operator,0,0,0,1


In [ ]:
final_data['COUNTER'] = final_data['COUNTER'].fillna('ONLTC') # for COUNTER

In [ ]:
final_data['R_CENTRE'] = final_data['R_CENTRE'].fillna('ONLINE BUS STATION')# for R_CENTRE

In [ ]:
final_data['BKTYPE'] = final_data['BKTYPE'].fillna('E')# for BKTYPE

In [ ]:
final_data['USERNAME'] = final_data['USERNAME'].fillna('ONLINE USER')# for USERNAME

In [ ]:
final_data['USERTYPE'] = final_data['USERTYPE'].fillna('Operator')# for USERTYPE

In [53]:
final = final_data.dropna() # drop the entire row if any one of the column has missing value  

# <brc>

## 3. Feature Engineering using one hot encoding 

In [ ]:
# Empty list to store columns with categorical data
categorical_final = []
for col, value in final.iteritems():
    if value.dtype == 'object':
        categorical_final.append(col)

# Store the numerical columns in a list numerical
numerical_final = final.columns.difference(categorical_final)

In [ ]:
# Store the categorical data in a dataframe called attrition_cat
final_cat = final[categorical_final]

In [54]:
final_cat = pd.get_dummies(final_cat)
final_cat.head(3)


,COUNTER_EKMC1,COUNTER_MFABUBS,COUNTER_MFAPKBS,COUNTER_MFARNBS,COUNTER_MFBLRBS,COUNTER_MFBODBS,COUNTER_MFCBABS,COUNTER_MFCBEKVP,COUNTER_MFCHEPOO,COUNTER_MFCHETVT,...,USERNAME_VEDARANYAM VENKATARAMAN S,USERNAME_VELANKANNI U NATHIYA,USERNAME_VELANKANNI VADIVKKRACI TWO,USERNAME_VELLORE K SARAVANAN,USERNAME_VILLUPURA MANOBALA G,USERNAME_VIRTUAL MASTER FRANCHISEE,USERNAME_VIRUDHACHALAM SENTHILVELAN,USERNAME_VIRUDHUNAGAR AYYANESHWARAN K,USERTYPE_Franchise,USERTYPE_Operator
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# <brc>

## 4. Creating new features from Numerical data

In [ ]:
# Store the numerical features to a dataframe attrition_num
final_num = final[numerical_final]
final_num = final_num.drop(['cancelled_Ticket'], axis=1) # To remove the target column PerformanceRating 

In [ ]:
# Concat the two dataframes together columnwise
final1 = pd.concat([final_num, final_cat], axis=1)
final1.head(3)
target = final.loc[:,['cancelled_Ticket']]

# <brc>

## 5. Split the data to train & test 

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit

train, test, target_train, target_test = train_test_split(final1, target, train_size= 0.75,random_state=0);

# <brc>

## 6. SMOTE to oversample due to the skewness in target


In [ ]:
from imblearn.over_sampling import SMOTE

oversampler= SMOTE(random_state=0)
smote_train, smote_target = oversampler.fit_sample(train,target_train)

# <brc>

## 7. Implement ML Model 

### (i) Logistic Regression 

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model_logr = LogisticRegression() # import the logistic regression 

In [40]:
import time
start = time.time()

model_logr.fit(smote_train, smote_target) # fitting the train and target in model

end = time.time()
total = end-start
print("Fitting Time:%s" % total)

Fitting Time:38.99868106842041


In [31]:
y_predict = model_logr.predict(test)
print("Predictions finished")

Predictions finished


In [30]:
accuracy_score(target_test, y_predict) # shows the accuracy of the predicted data 

0.9190041057826562

# <brc>

### (ii) Random Forest Classifier


In [41]:
seed = 0   # We set our random seed to zero for reproducibility
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 800,
    'warm_start': True, 
    'max_features': 0.3,
    'max_depth': 9,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}

In [42]:
rf = RandomForestClassifier(**rf_params)

In [43]:
import time
start = time.time()

rf.fit(smote_train, smote_target)               # fitting the train,test data in model 
print("Fitting of Random Forest as finished")

end = time.time()
total = end-start
print("Fitting Time:%s" % total)

Fitting of Random Forest as finished
Fitting Time:3378.75848197937


In [44]:
rf_predictions = rf.predict(test)
print("Predictions finished")

Predictions finished


In [45]:
accuracy_score(target_test, rf_predictions) # To find the accuracy of the Prediction Model 

0.9399892171364586

# <brc>

## 8. Feature Ranking via the Random Forest


In [70]:
# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = final1.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = final1.columns.values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance Level',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')